Load cholec80 transforms and translate to different python module with same functionality. Also translate from yaml file to pandas pickle format

In [2]:
import sys
import copy
import pandas as pd

sys.path.append('../')

from utils import load_yaml, save_yaml

servers = load_yaml('../configs/servers.yml')
server = servers['local']

databases = load_yaml('../configs/cholec80_transforms.yml')
database = databases['databases'][0]

# files, prefix in db], transform {module, type, kwargs}, database {name, prefix}
database_df = pd.DataFrame(columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary'])

aug_transforms = [
    # {'module': 'torchvision.transforms', 'type': 'ConvertImageDtype', 'kwargs': {'dtype': torch.float32}},  # works, but kills worker (shared memory too small, might work on server) normalizes image already to [0., 1.]
    # {'module': 'torchvision.transforms', 'type': 'ColorJitter', 'kwargs': {'brightness': 0.5}}, # rework transforms here
    {'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.5}},      # GaussianBlur, 
    {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}},   # RandomVerticalFlip & Horizontal -> to comp sq
    {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}  # RandomErasing, ConvertImageDtype
]

# load transforms and convert them to other module's callables
key_dict = {
    'Crop': 'Crop',
    'Resize': 'Resize'
}

for db_idx, db in enumerate(databases['databases']):

    for t_idx, old_transforms in enumerate(db['transforms']):
        pre_transforms = []
        for old_transform in old_transforms:
            # 'Crop' -> 'crop', shape -> height, width, top_left_corner -> top, left
            # 'Resize' -> 'resize', dsize -> size
            transform = {}
            for key, value in old_transform.items():
                if key == 'Crop':
                    transform = {
                        'module': 'utils.transforms',
                        'type': 'Crop',
                        'kwargs': {
                            'top_left_corner': value['top_left_corner'],
                            'shape': value['shape'][::-1],
                            'order': 'chw'
                        }
                    }
                elif key == 'Resize':
                    transform = {
                        'module': 'torchvision.transforms',
                        'type': 'Resize',
                        'kwargs': {
                            'size': value['dsize'][::-1]
                        }
                    }
                    

                else:
                    raise ValueError('Key not known')

            pre_transforms.append(transform)                
        
        df_row = {
            'database': database['name'],
            'train': True,
            'file': {'name': database['videos']['files'][t_idx], 'path': database['videos']['prefix']}, 
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        }
        database_df = database_df.append(df_row, ignore_index=True)

database_df.to_pickle('../configs/cholec80_transforms.pkl')
print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

KeyError: 'transforms'

In [1]:
import pandas as pd
import torch

# create dummy df
database_df = pd.DataFrame(columns=['database', 'train', 'file', 'pre_transforms', 'aug_transforms', 'auxiliary'])

pre_transforms = [
    {'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}},
    {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}
]

aug_transforms = [
    # {'module': 'torchvision.transforms', 'type': 'ConvertImageDtype', 'kwargs': {'dtype': torch.float32}},  # works, but kills worker (shared memory too small, might work on server) normalizes image already to [0., 1.]
    # {'module': 'torchvision.transforms', 'type': 'ColorJitter', 'kwargs': {'brightness': 0.1, 'contrast': 0.1, 'saturation': 0.1, 'hue': 0.1}}, # rework transforms here
    {'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.1}},      # GaussianBlur, 
    {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}},    # RandomVerticalFlip & Horizontal
    {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}    # RandomErasing, ConvertImageDtype
    # {'module': 'torchvision.transforms', 'type': 'GaussianBlur', 'kwargs': {'kernel_size': 9, 'sigma': (0.1, 1.)}}
]

for i in range(4):
    database_df = database_df.append(
        {
            'database': 'cholec80',
            'train': True,
            'file': {
                'name': 'video0{}_short.mp4'.format(i+1),
                'path': 'sample_videos'
            },
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        },
        ignore_index=True
    )

# add test set
for i in range(2):
    database_df = database_df.append(
        {
            'database': 'cholec80',
            'train': False,
            'file': {
                'name': 'video0{}_short.mp4'.format(i+1),
                'path': 'sample_videos'
            },
            'pre_transforms': pre_transforms,
            'aug_transforms': aug_transforms,
            'auxiliary': {}
        },
        ignore_index=True
    )


database_df.to_pickle('../configs/cholec80_dummy_transforms.pkl')
print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

pre-transforms:
 [{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}}, {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}]
aug-transforms:
 [{'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.1}}, {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}}, {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}]


,database,train,file,pre_transforms,aug_transforms,auxiliary
0,cholec80,True,"{'name': 'video01_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
1,cholec80,True,"{'name': 'video02_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
2,cholec80,True,"{'name': 'video03_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
3,cholec80,True,"{'name': 'video04_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
4,cholec80,False,"{'name': 'video01_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
5,cholec80,False,"{'name': 'video02_short.mp4', 'path': 'sample_...","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}


In [2]:
import pandas as pd

database_df = pd.read_pickle('../configs/cholec80_transforms.pkl')

print('pre-transforms:\n', database_df.pre_transforms[0])
print('aug-transforms:\n', database_df.aug_transforms[0])
database_df

pre-transforms:
 [{'module': 'utils.transforms', 'type': 'Crop', 'kwargs': {'top_left_corner': [58, 142], 'shape': [554, 416], 'order': 'chw'}}, {'module': 'torchvision.transforms', 'type': 'Resize', 'kwargs': {'size': [480, 640]}}]
aug-transforms:
 [{'module': 'torchvision.transforms', 'type': 'RandomGrayscale', 'kwargs': {'p': 0.5}}, {'module': 'torchvision.transforms', 'type': 'RandomVerticalFlip', 'kwargs': {'p': 0.5}}, {'module': 'torchvision.transforms', 'type': 'RandomHorizontalFlip', 'kwargs': {'p': 0.5}}]


,database,train,file,pre_transforms,aug_transforms,auxiliary
0,cholec80,True,"{'name': 'video01.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
1,cholec80,True,"{'name': 'video02.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
2,cholec80,True,"{'name': 'video03.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
3,cholec80,True,"{'name': 'video04.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
4,cholec80,True,"{'name': 'video05.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
...,...,...,...,...,...,...
70,cholec80,True,"{'name': 'video17.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
71,cholec80,True,"{'name': 'video18.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
72,cholec80,True,"{'name': 'video19.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
73,cholec80,True,"{'name': 'video20.mp4', 'path': 'videos'}","[{'module': 'utils.transforms', 'type': 'Crop'...","[{'module': 'torchvision.transforms', 'type': ...",{}
